In [ ]:
%pip install git+https://github.com/facebookresearch/segment-anything.git

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
def show_image(image):
    if image.ndim > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(image)

sam_checkpoint = "/content/drive/MyDrive/sam_vit_h_4b8939.pth"
model_type = "vit_h"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device="cuda")
predictor = SamPredictor(sam)

input_box = np.array([0, 0, 256, 256])

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2)) 

In [ ]:
input_point = np.array([[128, 128]])
input_label = np.array([1])
# for i in range(224, 1095):
for i in range(49, 574):
    # image = cv2.imread('/content/drive/MyDrive/data/Blight/Corn_Blight (' + str(i) + ').JPG')
    image = cv2.imread('/content/drive/MyDrive/data/Gray_Leaf_Spot/Corn_Gray_Spot (' + str(i) + ').JPG')
    print(i)
    if(image is None):
        # image = cv2.imread('/content/drive/MyDrive/data/Blight/Corn_Blight (' + str(i) + ').jpg')
        image = cv2.imread('/content/drive/MyDrive/data/Gray_Leaf_Spot/Corn_Gray_Spot (' + str(i) + ').jpg')
        if(image is None):
            continue
    predictor.set_image(image)

    masks, _, _ = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        box=input_box,
        multimask_output=False,
    )

    mask = masks[0]
    mask = (mask > 0.5).astype(np.uint8)
    masked_image = cv2.bitwise_and(image, image, mask=mask)

    # cv2.imwrite('/content/drive/MyDrive/seg_data/Blight/Corn_Blight (' + str(i) + ').jpg', masked_image)
    cv2.imwrite('/content/drive/MyDrive/seg_data/Gray_Leaf_Spot/Corn_Gray_Spot (' + str(i) + ').jpg', masked_image)